This notebook is for merging openstreetmaps and google places data into one file for upload to the database.

In [179]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

In [180]:
# Read in dataframes
df_am = pd.read_csv("../../data_evaluation/osm_places/osm_places_amenity_cleaned.csv")
df_le = pd.read_csv("../../data_evaluation/osm_places/osm_places_leisure_cleaned.csv")
df_sh = pd.read_csv("../../data_evaluation/osm_places/osm_places_shop_cleaned.csv")
df_go = pd.read_csv("../../data_evaluation/google_places_api/google_places.csv")

In [181]:
# Add type to dfs
df_am['type'] = 'amenity'
df_le['type'] = 'leisure'
df_sh['type'] = 'shop'

# Subtype and rename address cols
df_am.rename({'amenity':'subtype', 'addr:city':'addr_city', 'addr:housenumber':'addr_housenumber', 'addr:postcode': 'addr_postcode',
       'addr:state':'addr_state', 'addr:street':'addr_street'}, axis=1, inplace=True)
df_le.rename({'leisure':'subtype', 'addr:city':'addr_city', 'addr:housenumber':'addr_housenumber', 'addr:postcode': 'addr_postcode',
       'addr:state':'addr_state', 'addr:street':'addr_street'}, axis=1, inplace=True)
df_sh.rename({'shop':'subtype', 'addr:city':'addr_city', 'addr:housenumber':'addr_housenumber', 'addr:postcode': 'addr_postcode',
       'addr:state':'addr_state', 'addr:street':'addr_street'}, axis=1, inplace=True)

In [182]:
df_am.columns

Index(['id', 'lat', 'lon', 'addr_city', 'addr_housenumber', 'addr_postcode',
       'addr_state', 'addr_street', 'subtype', 'cuisine', 'drive_through',
       'name', 'opening_hours', 'phone', 'website', 'outdoor_seating',
       'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'wheelchair',
       'email', 'diet:vegan', 'diet:vegetarian', 'drink:coffee', 'drink:tea',
       'contact:instagram', 'type'],
      dtype='object')

In [183]:
# Combine dataframes
df = pd.concat([df_am, df_le, df_sh])
df = df.set_index('id')
df['addr_postcode'] = df['addr_postcode'].astype('Int64')

In [184]:
df

,lat,lon,addr_city,addr_housenumber,addr_postcode,addr_state,addr_street,subtype,cuisine,drive_through,...,drink:liquor,drink:wine,wheelchair,email,diet:vegan,diet:vegetarian,drink:coffee,drink:tea,contact:instagram,type
id,,,,,,,,,,,,,,,,,,,,,
349323821,40.762515,-73.976690,New York,43,10019,NY,West 55th Street,fast_food,burger,no,...,no,no,no,NaN,no,no,no,no,NaN,amenity
357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365,NY,190th Street,restaurant,burger,NaN,...,no,no,no,NaN,no,no,no,no,NaN,amenity
357620442,40.741558,-73.978463,NaN,NaN,<NA>,NaN,NaN,restaurant,korean,NaN,...,yes,yes,no,NaN,no,no,no,no,NaN,amenity
357621192,40.846563,-73.938127,New York,4140,10033,NY,Broadway,theatre,NaN,NaN,...,no,no,no,NaN,no,no,no,no,NaN,amenity
368042980,40.721480,-73.997103,NaN,97,10012,NaN,Kenmare Street,arts_centre,NaN,NaN,...,no,no,no,NaN,no,no,no,no,NaN,amenity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897341759,40.722718,-74.004212,NaN,NaN,<NA>,NaN,NaN,bakery,NaN,NaN,...,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,shop
11897355413,40.722687,-74.004612,NaN,NaN,<NA>,NaN,NaN,deli,NaN,NaN,...,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,shop
11920292910,40.729140,-73.953671,NaN,854,<NA>,NaN,Manhattan Avenue,deli,NaN,NaN,...,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,shop


In [185]:
# Rename some google col names
df_go.rename({'google_name':'google_id', 'lat':'google_lat', 'lon':'google_lon'}, axis=1, inplace=True)

In [186]:
df.columns

Index(['lat', 'lon', 'addr_city', 'addr_housenumber', 'addr_postcode',
       'addr_state', 'addr_street', 'subtype', 'cuisine', 'drive_through',
       'name', 'opening_hours', 'phone', 'website', 'outdoor_seating',
       'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'wheelchair',
       'email', 'diet:vegan', 'diet:vegetarian', 'drink:coffee', 'drink:tea',
       'contact:instagram', 'type'],
      dtype='object')

In [187]:
# Join dataframes
df = df.join(df_go.set_index('id'), on=['id'], how='inner')

In [188]:
df

,lat,lon,addr_city,addr_housenumber,addr_postcode,addr_state,addr_street,subtype,cuisine,drive_through,...,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
id,,,,,,,,,,,,,,,,,,,,,
349323821,40.762515,-73.976690,New York,43,10019,NY,West 55th Street,fast_food,burger,no,...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...
357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365,NY,190th Street,restaurant,burger,NaN,...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...
357620442,40.741558,-73.978463,NaN,NaN,<NA>,NaN,NaN,restaurant,korean,NaN,...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...
357621192,40.846563,-73.938127,New York,4140,10033,NY,Broadway,theatre,NaN,NaN,...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...
368042980,40.721480,-73.997103,NaN,97,10012,NaN,Kenmare Street,arts_centre,NaN,NaN,...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4900466400,40.708752,-74.005271,NaN,64,10038,NaN,Fulton Street,books,NaN,NaN,...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...
4913377713,40.755677,-73.990684,NaN,612,<NA>,NaN,8th Avenue,tattoo,NaN,NaN,...,places/ChIJCzvmzqxZwokRSJVrXD7H_Lo/photos/AUc7...,places/ChIJCzvmzqxZwokRSJVrXD7H_Lo/photos/AUc7...,p

In [189]:
df.columns

Index(['lat', 'lon', 'addr_city', 'addr_housenumber', 'addr_postcode',
       'addr_state', 'addr_street', 'subtype', 'cuisine', 'drive_through',
       'name', 'opening_hours', 'phone', 'website', 'outdoor_seating',
       'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'wheelchair',
       'email', 'diet:vegan', 'diet:vegetarian', 'drink:coffee', 'drink:tea',
       'contact:instagram', 'type', 'google_id', 'disp_name', 'google_lat',
       'google_lon', 'photo_0', 'photo_1', 'photo_2', 'photo_3', 'photo_4',
       'photo_5', 'photo_6', 'photo_7', 'photo_8', 'photo_9'],
      dtype='object')

In [190]:
# Remove unwanted cols
df = df[['google_id', 'lat', 'lon', 'name', 'type', 'subtype', 'addr_city', 'addr_housenumber', 'addr_postcode',
       'addr_state', 'addr_street', 'opening_hours', 'website', 'photo_0', 'photo_1', 'photo_2', 'photo_3', 'photo_4',
       'photo_5', 'photo_6', 'photo_7', 'photo_8', 'photo_9']]

In [191]:
# Reset index to include 'id' as col
df.reset_index(inplace=True)

In [192]:
# Drop rows that don't have at least 1 photo
df = df.drop(df[df['photo_0'].isna()].index)

In [193]:
df

,id,google_id,lat,lon,name,type,subtype,addr_city,addr_housenumber,addr_postcode,...,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
0,349323821,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc,40.762515,-73.976690,Five Guys,amenity,fast_food,New York,43,10019,...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...
1,357618253,places/ChIJcdatubBhwokRNd0ySdodDgg,40.740822,-73.784295,Hooters,amenity,restaurant,Fresh Meadows,61-09,11365,...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...
2,357620442,places/ChIJG8WWttZZwokRBkeFkHkoFv4,40.741558,-73.978463,Sam Sunny,amenity,restaurant,NaN,NaN,<NA>,...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...
3,357621192,places/ChIJ00nIlp_2wokROYs-HVvAfj8,40.846563,-73.938127,United Palace,amenity,theatre,New York,4140,10033,...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...
4,368042980,places/ChIJeaNd3IhZwokRU25epwuJ4aY,40.721480,-73.997103,Storefront for Art and Architecture,amenity,arts_centre,NaN,97,10012,...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,4900466400,places/ChIJhb4pSz1awokRMqAUarOuIB8,40.708752,-74.005271,Midtown Comics Downtown,shop,books,NaN,64,10038,...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...,places/ChIJhb4pSz1awokRMqAUarOuIB8/photos/AUc7...
219

In [194]:
df.to_csv("places.csv", index=False)